#### CSCE 670 :: Information Storage and Retrieval :: Texas A&M University :: Spring 2018


# Homework 3:  Embeddings + Recommenders

### 100 points [5% of your final grade]

### Due: Monday, April 9 by 11:59pm

*Goals of this homework:* There are two main learning objectives: (i) implement and evaluate a pre-cursor to modern word2vec embeddings; and (ii) implement, evaluate, and improve upon traditional collaborative filtering recommenders.

*Submission Instructions:* To submit your homework, rename this notebook as UIN_hw#.ipynb. For example, this homework submission would be: YourUIN_hw3.ipynb. Submit this notebook via ecampus. Your notebook should be completely self-contained, with the results visible in the notebook. 

*Late submission policy:* For this homework, you may use up to three of your late days, meaning that no submissions will be accepted after Thursday, April 12 at 11:59pm.

# Part 1: Word Embeddings (50 points)
For this first part, we're going to implement a word embedding approach that is a bit simpler than word2vec. The key idea is to look at co-occurrences between center words and context words (somewhat like in word2vec) but without any pesky learning of model parameters.

If you're interested in a deeper treatment of comparing count vs. learned embeddings, take a look at: [Don’t count, predict! A systematic comparison of
context-counting vs. context-predicting semantic vectors](
http://www.aclweb.org/anthology/P14-1023)

## Load the Brown Corpus

The dataset for this part is the (in)famous [Brown corpus](https://en.wikipedia.org/wiki/Brown_Corpus) that is a collection of text samples from a wide range of sources, with over one million unique words. Good for us, you can find the Brown corpus in nltk. *Make sure you have already installed nltk with something like: conda install nltk*

In [187]:
import nltk
nltk.download('brown')

[nltk_data] Downloading package brown to /Users/guanlun/nltk_data...
[nltk_data]   Package brown is already up-to-date!


True

Once you have it locally, you can load the dataset into your notebook. You can access the words using brown.words():

In [188]:
from nltk.corpus import brown
brown.words()

[u'The', u'Fulton', u'County', u'Grand', u'Jury', ...]

## 1.1 Dataset Pre-processing
OK, now we need to do some basic pre-processing. For this part you should:

* Remove stopwords and punctuation.
* Make everything lowercase.

Then, count how often each word occurs. We will define the 5,000 most  frequent words as your vocabulary (V). We will define the 1,000 most frequent words as our context (C). Include a print statement below to show the top-20 words after pre-processing.

In [189]:
# Your Code Here...
from nltk.corpus import stopwords
import re

vocabulary_size = 5000
context_size = 1000

en_stopwords = stopwords.words('english')

word_count = dict()

words = []

for word in brown.words():
    word_base = re.sub(r'\W+', '', word.lower())
    if len(word_base) != 0 and not word_base in en_stopwords:
        if word_base in word_count:
            word_count[word_base] += 1
        else:
            word_count[word_base] = 1
            
        words.append(word_base)
            
items = word_count.items()
items.sort(key=lambda w : -w[1])

vocabulary = [w[0] for w in items[0:vocabulary_size]]
context = [w[0] for w in items[0:context_size]]

for i in xrange(20):
    print str(i + 1) + "\t" + items[i][0] + "\t" + str(items[i][1])

1	one	3297
2	would	2714
3	said	1961
4	new	1635
5	could	1601
6	time	1598
7	two	1412
8	may	1402
9	first	1361
10	like	1292
11	man	1207
12	even	1170
13	made	1125
14	also	1069
15	many	1030
16	must	1013
17	years	1001
18	af	996
19	back	966
20	well	961


## 1.2 Building the Co-occurrence Matrix 

For each word in the vocabulary (w), we want to calculate how often context words from C appear in its surrounding window of size 4 (two words before and two words after).

In other words, we need to define a co-occurrence matrix that has a dimension of |V|x|C| such that each cell (w,c) represents the number of times c occurs in a window around w. 

In [220]:
# Your Code Here...
import numpy as np

co_occurrence = np.zeros((vocabulary_size, context_size))

idx_lookup = dict()

for idx, word in enumerate(vocabulary):
    idx_lookup[word] = idx

for idx, word in enumerate(words):
    if word in vocabulary:
        neighbors = words[idx - 2: idx] + words[idx + 1: idx + 3]

        voc_idx = idx_lookup[word]

        for n in neighbors:
            if n in context:
                ctx_idx = idx_lookup[n]
                co_occurrence[voc_idx, ctx_idx] += 1
                
print co_occurrence

[[ 82.  76.  52. ...,   2.   1.   0.]
 [ 76.  70.  68. ...,   2.   1.   0.]
 [ 52.  68.  34. ...,   4.   1.   0.]
 ..., 
 [  0.   1.   0. ...,   0.   0.   0.]
 [  0.   0.   0. ...,   0.   0.   0.]
 [  0.   0.   0. ...,   0.   0.   0.]]


## 1.3 Probability Distribution

Using the co-occurrence matrix, we can compute the probability distribution Pr(c|w) of context word c around w as well as the overall probability distribution of each context word c with Pr(c).  

In [221]:
# Your Code Here...
probability_mtx = np.zeros((vocabulary_size, context_size))

context_words_count = co_occurrence.sum(axis=0)

for ctx_idx in xrange(context_size):
    for voc_idx in xrange(vocabulary_size):
        probability_mtx[voc_idx, ctx_idx] = co_occurrence[voc_idx, ctx_idx] / context_words_count[ctx_idx]

total_context_words_count = context_words_count.sum()
print probability_mtx

context_words_probability = context_words_count / total_context_words_count
print context_words_probability

[[ 0.00812122  0.00902184  0.00880759 ...,  0.00657895  0.00327869  0.        ]
 [ 0.00752699  0.00830959  0.01151762 ...,  0.00657895  0.00327869  0.        ]
 [ 0.00515004  0.00807217  0.00575881 ...,  0.01315789  0.00327869  0.        ]
 ..., 
 [ 0.          0.00011871  0.         ...,  0.          0.          0.        ]
 [ 0.          0.          0.         ...,  0.          0.          0.        ]
 [ 0.          0.          0.         ...,  0.          0.          0.        ]]
[ 0.01379023  0.01150529  0.00806354  0.00692038  0.00692175  0.00693677
  0.00589332  0.00593566  0.00585917  0.00506566  0.00509024  0.00477338
  0.00459856  0.00430492  0.00438687  0.00438004  0.00447291  0.00424619
  0.00407957  0.00413557  0.00407547  0.00383919  0.00374905  0.00310714
  0.00358106  0.00346907  0.00350868  0.00349229  0.0035155   0.00337893
  0.00323142  0.00337483  0.00323552  0.00324918  0.00313445  0.00338166
  0.00303475  0.00302383  0.00289135  0.0028613   0.00311397  0.00294871
 

## 1.4 Embedding Representation

Now you can represent each vocabulary word as a |C| dimensional vector using this equation:

Vector(w)= max(0, log (Pr(c|w)/Pr(c)))

This is a traditional approach called *pointwise mutual information* that pre-dates word2vec by some time. 

In [222]:
# Your Code Here...

vocabulary_vectors = np.zeros((vocabulary_size, context_size))

for voc_idx in xrange(vocabulary_size):
    for ctx_idx in xrange(context_size):
        context_relative_probability = probability_mtx[voc_idx, ctx_idx] / context_words_probability[ctx_idx]
        if context_relative_probability < 1:
            vocabulary_vectors[voc_idx, ctx_idx] = 0
        else:
            vocabulary_vectors[voc_idx, ctx_idx] = math.log(context_relative_probability)
            
print vocabulary_vectors

[[ 0.          0.          0.08826148 ...,  2.76288027  2.06316494  0.        ]
 [ 0.          0.          0.35652547 ...,  2.76288027  2.06316494  0.        ]
 [ 0.          0.          0.         ...,  3.45602745  2.06316494  0.        ]
 ..., 
 [ 0.          0.          0.         ...,  0.          0.          0.        ]
 [ 0.          0.          0.         ...,  0.          0.          0.        ]
 [ 0.          0.          0.         ...,  0.          0.          0.        ]]


## 1.5 Analysis

So now we have some embeddings for each word. But are they meaningful? For this part, you should:

- First, cluster the vocabulary into 100 clusters using k-means. Look over the words in each cluster, can you see any relation beween words? Discuss your observations.

- Second, for the top-20 most frequent words, find the nearest neighbors using cosine distance (1- cosine similarity). Do the findings make sense? Discuss.

In [223]:
# Your Code Here...

from sklearn.cluster import KMeans

cluster_count = 100

buckets = []
for i in xrange(cluster_count):
    buckets.append([])

kmeans = KMeans(n_clusters=cluster_count)
kmeans.fit(vocabulary_vectors)
cluster_results = kmeans.labels_

for i in xrange(vocabulary_size):
    bucket_idx = cluster_results[i]
    voc_word = vocabulary[i]
    
    buckets[bucket_idx].append(voc_word)

for bucket in buckets:
    print bucket

[u'less']
[u'found', u'school', u'war', u'though', u'almost', u'enough', u'yet', u'better', u'later', u'group', u'young', u'often', u'early', u'need', u'children', u'best', u'church', u'least', u'others', u'family', u'although', u'done', u'kind', u'different', u'perhaps', u'quite', u'words', u'already', u'seems', u'true', u'usually', u'strong', u'book']
[u'matter']
[u'put', u'per', u'four', u'times', u'five', u'money', u'probably', u'brought', u'ago', u'full', u'six', u'million', u'minutes', u'months', u'hours', u'10', u'miles', u'added', u'pay', u'hundred', u'ten', u'working', u'weeks', u'month', u'seven', u'dollars', u'thousand']
[u'us', u'let']
[u'look', u'felt', u'ever', u'want', u'mind', u'god', u'help', u'name', u'anything', u'seen', u'really', u'word', u'tell', u'sure', u'real', u'miss', u'heard', u'reason', u'love', u'wife', u'voice', u'wanted', u'woman', u'ill', u'girl', u'mother', u'feel', u'child', u'leave', u'hard', u'play', u'believe', u'says', u'mean', u'gone', u'idea', u

Some of the clusters made sense. For example, each of these word lists represents one cluster:
* __Names and Titles__: mrs, president, john, c, dr, b, brown, william, p, george, e, j, thomas, james, director, r, f, h, w, robert, henry, l, jr, chairman, g, joseph, smith
* __Common verbs__: get, come, go, home, say, dont, away, think, told, didnt, find, knew, im
* __Somewhat Related to government__: united, government, city, national, development, members, service, york, company, local, act, college, special, federal, board, court, department, party, nations, university, education, military, class, washington, plan, schools, secretary, section, union, recent, report, research, committee, defense, island, central, received, medical, administration, meeting, foreign, training, international, congress, county, countries, labor, trade, industrial, programs, kennedy, services, member, aid, district, association, army, student, planning, organization, reported, responsibility, activities, plans, leaders, staff, corps, democratic, division, health, rhode, council, commission, officers, conference, citizens, assistance

However, some of the clusters contains words that does not appear to have obviously similar meanings. In addition, there are a few huge clusters with along with some other clusters with only one word.

# Part 2. Collaborative Filtering (50 points)

In this second part, you will implement collaborative filtering on the Netflix prize dataset -- don’t freak out, the provided sample dataset has only ~2000 items and ~28,000 users.

As background, read the paper [Empirical Analysis of Predictive Algorithms for Collaborative Filtering](https://arxiv.org/pdf/1301.7363.pdf) up to Section 2.1. Of course you can read further if you are interested, and you can also refer to the course slides for collaborative filtering.

## 2.1 Load Netflix Data

The dataset is subset of movie ratings data from the Netflix Prize Challenge. Download the dataset from Piazza. It contains a train set, test set, movie file, and README file. The last two files are original ones from the Netflix Prize, however; in this homework you will deal with train and test files which both are subsets of the Netflix training data. Each of train and test files has lines having this format: MovieID,UserID,Rating.

Your job is to predict a rating in the test set using those provided in the training set.

In [2]:
# load the data, then print out the number of ratings, 
# movies and users in each of train and test sets.
# Your Code Here...

import numpy as np

# create two-way lookups so that we can later find the index given its ID in the rating matrix, and vice versa
movie_ids = []
movie_id_lookup = {}

user_ids = []
user_id_lookup = {}

def read_rating_file(filename):
    rating_matrix = None
    
    with open(filename) as training_file:
        records = []
        
        movie_idx = 0
        user_idx = 0

        for idx, line in enumerate(training_file):
            segments = line.rstrip("\r\n").split(',')
            movie_id = int(segments[0])
            user_id = int(segments[1])
            rating = float(segments[2])
            
            if movie_id not in movie_id_lookup:
                movie_id_lookup[movie_id] = movie_idx
                movie_ids.append(movie_id)
                movie_idx += 1
                
            if user_id not in user_id_lookup:
                user_id_lookup[user_id] = user_idx
                user_ids.append(user_id)
                user_idx += 1

            records.append((movie_id, user_id, rating))
    
        movie_count = len(movie_ids)
        user_count = len(user_ids)
        
        rating_matrix = np.zeros((movie_count, user_count))
        
        for record in records:
            movie_idx = movie_id_lookup[record[0]]
            user_idx = user_id_lookup[record[1]]
            
            rating_matrix[movie_idx, user_idx] = record[2]

        return records, rating_matrix

training_records, training_rating_mtx = read_rating_file('./netflix-dataset/TrainingRatings.txt')
testing_records, testing_rating_mtx = read_rating_file('./netflix-dataset/TestingRatings.txt')

## 2.2 Implement CF

In this part, you will implement the basic collaborative filtering algorithm described in Section 2.1 of the paper -- that is, focus only on Equations 1 and 2 (where Equation 2 is just the Pearson correlation). You should consider the first 5,000 users with their associated items in the test set. 

Note that you should test the algorithm for a small set of users e.g., 10 users first and then run for 5,000 users. It may take long to run but you won't have memory issues. 

Set k to 0.1. 

In [183]:
# This is the initial step for the collaborative filtering algorithm.
# Here I computed the mean rating for each user and will use them in the following parts

import math

total_movie_count = training_rating_mtx.shape[0]
total_user_count = training_rating_mtx.shape[1]

user_corelation_mtx = np.zeros((total_user_count, total_user_count))

def user_mean(user_idx):
    user_total_rating = 0
    user_rated_count = 0
    for j in xrange(movie_count):
        rating = training_rating_mtx[j, user_idx]
        if rating != 0:
            user_total_rating += rating
            user_rated_count += 1
            
    return user_total_rating / user_rated_count

user_mean_ratings = []

for user_idx in xrange(total_user_count):
    user_mean_ratings.append(user_mean(user_idx))

In [184]:
# In this step, I computed which movies each user rated and stored them into sets.
# Later when computing the corelation matrix, there's not need to scan through all
# the movies to find which were rated by both users, a simple intersection operation
# would find the common movies.

user_rated = []

movie_count = training_rating_mtx.shape[0]
user_count = training_rating_mtx.shape[1]

for i in xrange(user_count):
    rated_movies = []
    for j in xrange(movie_count):  
        if training_rating_mtx[j, i] != 0:
            rated_movies.append(j)

    user_rated.append(set(rated_movies))

In [178]:
##################################################################
##### DO NOT EXECUTE THIS CELL: IT WILL TAKE HOURS TO FINISH #####
##################################################################

# Here I computed the corelation matrix for the first 5000 users in the testing dataset.
# The result was stored in a user x user matrix where each element represents the corelation of two users

target_user_indices = [user_id_lookup[r[1]] for r in testing_records[0:5000]]

for idx, user_a_idx in enumerate(target_user_indices):
    if idx % 100 == 0:
        print idx
    user_a_mean = user_mean_ratings[user_a_idx]
    
    for user_i_idx in xrange(total_user_count):
        user_i_mean = user_mean_ratings[user_i_idx]
        
        common_rated_movie_indices = user_rated[user_a_idx].intersection(user_rated[user_i_idx])
        
        multiplied_sum = 0
        user_a_sum = 0
        user_i_sum = 0
                
        for common_movie_idx in common_rated_movie_indices:
            user_a_diff = training_rating_mtx[common_movie_idx, user_a_idx] - user_a_mean
            user_i_diff = training_rating_mtx[common_movie_idx, user_i_idx] - user_i_mean
            
            multiplied_sum += user_a_diff * user_i_diff
            user_a_sum += user_a_diff * user_a_diff
            user_i_sum += user_i_diff * user_i_diff
            
        w = 0
        if user_a_sum * user_i_sum != 0:
            w = multiplied_sum / math.sqrt(user_a_sum * user_i_sum)
            
        user_corelation_mtx[user_a_idx, user_i_idx] = w

0


KeyboardInterrupt: 

In [150]:
prediction_results = []

for idx, testing_record in enumerate(testing_records[0:5000]):
    if idx % 100 == 0:
        print idx
    movie_idx = movie_id_lookup[testing_record[0]]
    user_a_idx = user_id_lookup[testing_record[1]]
    
    vote_diff_sum = 0
    sim_abs_sum = 0

    for user_i_idx in xrange(total_user_count):
        user_i_rating = training_rating_mtx[movie_idx, user_i_idx]
        
        if user_i_idx == user_a_idx or user_i_rating == 0:
            continue
        
        w = user_corelation_mtx[user_a_idx, user_i_idx]
        sim_abs_sum += math.fabs(w)
        
        vote_diff_sum += w * (user_i_rating - user_mean_ratings[user_i_idx])

    rating_prediction = 0
    if sim_abs_sum == 0:
        rating_prediction = user_mean_ratings[user_a_idx]
    else:
        rating_prediction = user_mean_ratings[user_a_idx] + vote_diff_sum / sim_abs_sum
    prediction_results.append(rating_prediction)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900


## 2.3 Evaluation 

You should evaluate your predictions using Mean Absolute Error and Root Mean Squared Error. 

In [151]:
# Mean Absolute Error
import math

def compute_mean_absolute_error(predicted, actual):
    error_sum = 0

    prediction_count = len(predicted)

    for i in xrange(prediction_count):
#         print predicted[i], math.fabs(actual[i] - predicted[i])
        error_sum += math.fabs(actual[i] - predicted[i])

    mean_abs_error = error_sum / prediction_count
    return mean_abs_error

print 'Mean absolute error:', compute_mean_absolute_error(prediction_results, [r[2] for r in testing_records])

Mean absolute error: 0.692905886274


In [152]:
# Root Mean Squared Error

import math

def compute_rmse(predicted, actual):
    error_sum = 0

    prediction_count = len(predicted)

    for i in xrange(prediction_count):
        error_sum += math.pow(actual[i] - predicted[i], 2)

    rmse = math.sqrt(error_sum / prediction_count)
    return rmse

print 'Root Mean Squared Error:', compute_rmse(prediction_results, [r[2] for r in testing_records])

Root Mean Squared Error: 0.887790924116


## 2.4 Extensions

Given your results in the previous part, can you do better? For this last part you should report on your best attempt at improving MAE and RMSE. Provide code, results, plus a brief discussion on your approach.

In [177]:
extended_prediction_results = []

for idx, testing_record in enumerate(testing_records[0:5000]):
    if idx % 100 == 0:
        print idx
    movie_idx = movie_id_lookup[testing_record[0]]
    user_a_idx = user_id_lookup[testing_record[1]]
    
    vote_diff_sum = 0
    sim_abs_sum = 0

    for user_i_idx in xrange(total_user_count):
        user_i_rating = training_rating_mtx[movie_idx, user_i_idx]
        
        if user_i_idx == user_a_idx or user_i_rating == 0:
            continue
        
        w = user_corelation_mtx[user_a_idx, user_i_idx]
        sim_abs_sum += math.fabs(w)
        
        vote_diff_sum += w * (user_i_rating - user_mean_ratings[user_i_idx])

    rating_prediction = 0
    if sim_abs_sum == 0:
        rating_prediction = user_mean_ratings[user_a_idx]
    else:
        rating_prediction = user_mean_ratings[user_a_idx] + vote_diff_sum / sim_abs_sum * 1.3
    
    extended_prediction_results.append(rating_prediction)

# Mean Absolute Error
print 'Mean absolute error:', compute_mean_absolute_error(extended_prediction_results, [r[2] for r in testing_records])

# Root Mean Squared Error
print 'Root Mean Squared Error:', compute_rmse(extended_prediction_results, [r[2] for r in testing_records])

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
Mean absolute error: 0.687437364861
Root Mean Squared Error: 0.883212624709


I simply increased the normalizing factor *k* by 30% and got slightly better results.

When testing the basic collaborative filter algorithm in 2.2, I noticed that for most cases when the actual user rating deviated from the user's average rating by a large amount, even after adding the collaborative term the predicted result was still relatively far away from the actual rating.

Therefore I thought probably increasing the coeffcient on the collaborative term could bring the predicted results closer to the actual results. After trying different values (from 10% to 50%) I selected 30%, i.e. 1.3 times the regular computed normalizing factor.

The results were 0.8% more accurate compare to the method in 2.2 in terms of mean absolute error and 0.5% more accurate for RMSE.